# Installation

In [ ]:
# Mount gdrive for data sets
import os
from google.colab import drive
drive.mount('/gdrive')
try:
  os.symlink('/gdrive/My Drive/PHD/Tez', '/drive')
except FileExistsError:
  print("Seems like already mounted the gdrive, skipping the mounting part.")

# Download source code and copy to mounted drive
!rm -f master.zip
!wget https://github.com/aligokalppeker/hypelcnn/archive/refs/heads/master.zip
!unzip -d /drive/ -o -q master.zip && rm master.zip
!rm -rf /drive/hypelcnn/ && mv /drive/hypelcnn-master/ /drive/hypelcnn/

# Add source root to python PATH
import os
os.environ["PYTHONPATH"] += ":/drive/hypelcnn"
!echo $PYTHONPATH

# Package setup
!apt-get --assume-yes install libcupti-dev
!pip install -r /drive/hypelcnn/requirements.txt

# CNN Training and Visualization

In [ ]:
!python -B /drive/hypelcnn/classify/train_for_classification.py \
      --path=/drive/data \
      --loader_name=GULFPORTALTDataLoader \
      --importer=GeneratorImporter \
      --step=90000 \
      --max_evals=1 \
      --split_count=1 \
      --test_ratio=0 \
      --neighborhood=3 \
      --model_name=CONCNNModel \
      --algorithm_param_path=/drive/hypelcnn/nnmodel/modelconfigs/alg_param_concnn.json \
      --perform_validation \
      --save_checkpoint_steps=10000 \
      --base_log_path=/drive/class_log \
      --validation_steps=10000 \
      --train_ratio=125 \
      --augmentation_random_threshold=0.50 \
      --augment_data_with_shadow=simple

In [ ]:
!python -B /drive/hypelcnn/classify/infer_for_classification.py \
      --path=/drive/data \
      --loader_name=GULFPORTALTDataLoader \
      --importer=GeneratorImporter \
      --model_name=CONCNNModel \
      --neighborhood=3 \
      --batch_size=4000 \
      --base_log_path=/drive/class_log/episode_0/run_0/model.ckpt-0 \
      --output_path=/drive/class_log \
      --algorithm_param_path=/drive/hypelcnn/nnmodel/modelconfigs/alg_param_concnn.json

# Classic ML Tests

In [ ]:
!python -B /drive/hypelcnn/classify/classic_ml_trainer.py \
      --path=/drive/data \
      --loader_name=GULFPORTDataLoader \
      --neighborhood=1 \
      --batch_size=20000 \
      --split_count=1

# Learn Shadow via GAN

In [ ]:
!python -B /drive/hypelcnn/gan/gan_train_for_shadow.py \
      --path=/drive/data \
      --base_log_path=/drive/gan_log \
      --pairing_method=random \
      --step=100000 \
      --loader_name=GRSS2013DataLoader \
      --batch_size=32 \
      --validation_steps=1000 \
      --use_identity_loss=False \
      --cycle_consistency_loss_weight=10 \
      --gan_type=cycle_gan \
      --regularization_support_rate=0.0

In [ ]:
!python -B /drive/hypelcnn/gan/gan_infer_image_for_shadow.py \
      --path=/drive/data \
      --base_log_path=/drive/gan_log/model.ckpt-0 \
      --output_path=/drive/gan_log \
      --loader_name=GRSS2013DataLoader \
      --make_them_shadow=shadow \
      --convert_all=False \
      --gan_type=cycle_gan

In [ ]:
!python -B /drive/hypelcnn/gan/gan_train_for_shadow.py \
      --path=/drive/data \
      --base_log_path=/drive/gan_log \
      --pairing_method=random \
      --step=40000 \
      --loader_name=GRSS2013DataLoader \
      --batch_size=32 \
      --validation_steps=1000 \
      --use_identity_loss=True \
      --gan_type=dcl_gan \
      --regularization_support_rate=0.0 \
      --flag_config_file_min=./gan/flags_min.json \
      --flag_config_file_max=./gan/flags_max.json \
      --opt_run_count=3 \
      --opt_trial_count=20